# Figueroa LAML HM450 probes clustering analysis

Load subset of probes filtered from Figueroa paper and display the heatmap

In [3]:
library(pheatmap)
library(RColorBrewer)
library(viridis)
library(dplyr)
source("../R/regressionModel/regressionModel.R")

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-13

Loading required package: grid
ComplexHeatmap version 1.17.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://bioconductor.org/packages/ComplexHeatmap/

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.



In [ ]:
Figueroa.HM450 <- read.csv('../data/LAML/processed/HM450-imputed-matched.tsv', sep='\t', stringsAsFactor=F, header=F)
HM450 <- data.matrix(Figueroa.HM450[-1,-1])
rownames(HM450) <- Figueroa.HM450[-1,1]
colnames(HM450) <- Figueroa.HM450[1,-dim(Figueroa.HM450)[2]]
head(HM450)
dim(HM450)

## try different number of clusters and xx probes are the best
see if the results are similar as on HM27 array (which has 1470 probes) ? compare with lasso

In [ ]:
n = 

In [ ]:
options(repr.plot.width=8, repr.plot.height=12)

res <- pheatmap(
  mat               = HM450,
  color             = inferno(10),
  border_color      = NA,
  show_colnames     = FALSE,
  show_rownames     = FALSE,
  fontsize          = 14,
  cutree_rows       = n,
  main              = "All probes"
)

In [ ]:
res.clust <- cbind(HM450, 
                   cluster = cutree(res$tree_row, k = n))
head(res.clust)

In [ ]:
table(res.clust[,dim(res.clust)[2]])

## Load RNAseq data

In [ ]:
rnaseq.raw <- read.csv('../data/LAML/processed/rnaseq-imputed-matched-filtered.tsv', sep='\t', stringsAsFactor=F, header=F)
rnaseq <- data.matrix(rnaseq.raw[-1, -1])
colnames(rnaseq) <- rnaseq.raw[1, -dim(rnaseq.raw)[2]]
rownames(rnaseq) <- rnaseq.raw[-1, 1]
head(rnaseq)  # predictors

## Compute average methylation level functions and set the training set (compute training set and testing set RMSE)

In [ ]:
findClusterIdx <- function(i) {
    n <- dim(res.clust)[2]
    which(res.clust[,n] == i)
}

In [ ]:
computeAverage <- function(idx) {
    n <- dim(res.clust)[2]
    cluster.probes <- res.clust[idx, -n]
    cbind(apply(cluster.probes, 2, mean), apply(cluster.probes, 2, sd))
}

In [ ]:
set.seed(1)
train.idx <- sample(colnames(rnaseq), round(dim(rnaseq)[2] * 0.75, 0)) # 75 % as training set

In [ ]:
rnaseq.train <- rnaseq[, colnames(rnaseq) %in% train.idx]
head(rnaseq.train)

In [ ]:
findIdx <- function(gene) {
    which(rownames(rnaseq) %in% gene)
}
FindIdx <- Vectorize(findIdx)

In [ ]:
predict.vbsr <- function(vbsr.fit, new.rnaseq, y.true) {
    alpha <- vbsr.fit$alpha
    beta <- vbsr.fit$beta
    y.hat <- alpha + t(new.rnaseq) %*% matrix(beta, ncol=1)
    rmse <- sqrt(mean((y.true - y.hat)^2))
    return(rmse)
}

In [ ]:
n <- dim(HM450)[2]
plotCluster <- function(i) {
    set.seed(1)
    idx <- findClusterIdx(i)
    cluster.probes <- HM27[idx, -n]
    train.rnaseq.idx <- colnames(rnaseq) %in% train.idx
    train.hm450.idx <- colnames(HM450) %in% train.idx
    
    cluster.mean <- computeAverage(findClusterIdx(i))
    
    model.fit <- vbsr(y=cluster.mean[train.hm450.idx], X=t(rnaseq[,train.rnaseq.idx]), 
                      family='normal', eps=5e-15, maxit=10^4, post=0.5)
    rmse <- predict.vbsr(model.fit, rnaseq[,-train.rnaseq.idx], cluster.mean[-train.hm450.idx])
    
    post <- model.fit$post
    significant.idx <- post > 0.3
    coef.names <- rownames(rnaseq.train)[significant.idx]
    coef.significant <- model.fit$beta[significant.idx]
    res1 <- data.frame(gene=coef.names, coef=coef.significant)
    print(res1)
    ModuleHeatmap(t(cluster.probes), cluster.mean[,1], t(rnaseq[as.numeric(FindIdx(res1$gene)), ] * as.vector(sign(res1$coef))), center=T, scale=T)
    return(rmse)
}

## VBSR analysis

In [ ]:
for (i in seq(n)) {
    plotCluster(i)
}